In [ ]:
import numpy as np
import pandas as pd

Read the store data

In [ ]:
store = pd.read_csv("../input/store.csv")
logger.debug("Read store data: {0} rows, {1} columns".format(*store.shape))

if REDUCE_DATA:
    logger.debug("DROPPING DATA FOR DEBUGGING")
    store = store.iloc[:50, :]
    logger.debug("Store data reduced to {0} rows".format(store.shape[0]))

In [ ]:
def get_date(row):
    """
    Combines the year and month values to return a date
    :param row: DataFrame row
    :return: Timestamp for the date
    """
    year, month = row[['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth']]
    if not pd.isnull(year):
        return pd.Timestamp(int(year), int(month), 1)


store['CompetitionOpenDate'] = store.apply(get_date, axis=1)

In [ ]:
train_csv = pd.read_csv("../input/train.csv", low_memory=False)
logger.debug("Read train data: {0} rows, {1} columns".format(*train_csv.shape))

In [ ]:
train_csv['Open'] = train_csv['Sales'].apply(lambda s: 1 if s > 0 else 0)
assert check_nulls(train_csv, 'Open')
logger.debug("Patch train_csv Open")

train_csv['Date'] = pd.to_datetime(train_csv['Date'])
assert check_nulls(train_csv, 'Date')
logger.debug("Convert train_csv date to datetime")

In [ ]:
# Generate DatetimeIndex for the range of dates in the training set
train_range = pd.date_range(train_csv['Date'].min(), train_csv['Date'].max(), name='Date')
logger.debug("Training range is {0} - {1}".format(train_range.min(), train_range.max()))

In [ ]:
# Fill in gaps in dates for each store

def fill_gaps_by_store(g):
    s = g['Store'].iloc[0]
    logger.debug("Store {s} initial shape {shape}"
                 .format(s=s, shape=g.shape))
    filled = (g.set_index('Date')
              .reindex(train_range)
              .fillna(value={'Store': s, 'Sales': 0, 'Customers': 0, 'Open': 0,
                             'Promo': 0, 'StateHoliday': '0',
                             'SchoolHoliday': 0}))
    filled['DayOfWeek'] = filled.index.weekday + 1
    logger.debug("Store {s} shape after filling {shape}"
                 .format(s=s, shape=filled.shape))
    return filled.reset_index()


#  DayOfWeek: Monday is 0, Sunday is 6
logger.debug("Expand index of each store to cover full date range")
train_full = train_csv.groupby('Store').apply(fill_gaps_by_store).reset_index(drop=True)
assert check_nulls(train_full)
logger.debug("Expanded train data from shape {0} to {1}".format(train_csv.shape, train_full.shape))

In [ ]:
test_csv = pd.read_csv("../input/test.csv", low_memory=False)
logger.debug("Read test data: {0} rows, {1} columns".format(*test_csv.shape))

In [ ]:
test_csv['Open'].fillna(value=1, inplace=True)
assert check_nulls(test_csv, 'Open')
logger.debug("Fill nas test_csv Open")
test_csv['Date'] = pd.to_datetime(test_csv['Date'])
assert check_nulls(test_csv, 'Date')
logger.debug("Convert test_csv date to datetime")

In [ ]:
# Generate DatetimeIndex for the range of dates in the testing set
# and the full range over both training and test sets.

test_range = pd.date_range(test_csv['Date'].min(), test_csv['Date'].max(), name='Date')
logger.debug("Test data range is {0} - {1}".format(test_range.min(), test_range.max()))

full_range = pd.date_range(min(train_csv['Date'].min(), test_csv['Date'].min()),
                           max(train_csv['Date'].max(), test_csv['Date'].max()), name='Date')
logger.debug("Full data range is {0} - {1}".format(full_range.min(), full_range.max()))

In [ ]:
# Generate Fourier terms

def fourier(ts_length, period, terms):
    """
    Generate periodic terms with the given period
    :param ts_length: The length of the generated terms (corresponding to the
    time series length)
    :param period: The period
    :param terms: The number of series to generate.
    :return: DataFrame of shape ts_length, 2*terms. Columns are named
    s1, c1, s2, c2, ..., s<terms>, c<terms>
    """
    A = pd.DataFrame()
    fns = OrderedDict([('s', np.sin), ('c', np.cos)])
    terms_range = range(1, terms + 1)
    for term, (fn_name, fn) in product(terms_range, fns.items()):
        A[fn_name + str(term)] = fn(2 * np.pi * term * np.arange(1, ts_length + 1) / period)
    logger.debug("Created fourier terms of shape {0}".format(A.shape))
    return A

In [ ]:
fourier_terms = 5

fourier_features = fourier(len(full_range), period=365, terms=fourier_terms)
fourier_names = ['Fourier' + str(x) for x in range(1, 2 * fourier_terms + 1)]
fourier_features.columns = fourier_names
fourier_features['Date'] = full_range

In [ ]:
base_linear_features = ["Promo", "Promo2Active", "SchoolHoliday",
                        "DayOfWeek1", "DayOfWeek2", "DayOfWeek2", "DayOfWeek3",
                        "DayOfWeek4", "DayOfWeek5", "DayOfWeek6",
                        "StateHolidayA", "StateHolidayB", "StateHolidayC",
                        "CompetitionOpen", "Open"]

trend_features = ["DateTrend", "DateTrendLog"]

decay_features = ["PromoDecay", "Promo2Decay",
                  "StateHolidayCLastDecay", "StateHolidayCNextDecay",
                  "StateHolidayBNextDecay"]

log_decay_features = list(map("{0}Log".format, decay_features))

In [ ]:
stairs_steps = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 28)

stairs_features = chain(
    ["Opened", "PromoStarted", "Promo2Started", "TomorrowClosed",
     "WasClosedOnSunday"],
    map("PromoStartedLastDate{0}after".format, (2, 3, 4)),
    starmap("{1}{0}before".format,
            product(stairs_steps, ("StateHolidayCNextDate", "StateHolidayBNextDate",
                                   "StateHolidayANextDate", "LongClosedNextDate"))),
    starmap("{1}{0}after".format,
            product(stairs_steps, ("StateHolidayCLastDate", "StateHolidayBLastDate",
                                   "Promo2StartedDate", "LongOpenLastDate")))
)

In [ ]:
month_day_features = map("MDay{0}".format, range(1, 32))

month_features = map("Month{0}".format, range(1, 13))

In [ ]:
linear_features = chain(base_linear_features, trend_features, decay_features,
                        log_decay_features, stairs_features, fourier_names,
                        month_day_features, month_features)

glm_features = ("Promo", "SchoolHoliday",
                "DayOfWeek1", "DayOfWeek2", "DayOfWeek2", "DayOfWeek3",
                "DayOfWeek4", "DayOfWeek5", "DayOfWeek6",
                "StateHolidayA", "StateHolidayB", "StateHolidayC",
                "CompetitionOpen", "PromoDecay", "Promo2Decay",
                "DateTrend", "DateTrendLog",
                "StateHolidayCLastDecay", "StateHolidayCNextDecay",
                "Fourier1", "Fourier2", "Fourier3", "Fourier4")

log_lm_features = ("Promo", "Promo2Active", "SchoolHoliday",
                   "DayOfWeek1", "DayOfWeek2", "DayOfWeek2", "DayOfWeek3",
                   "DayOfWeek4", "DayOfWeek5", "DayOfWeek6",
                   "StateHolidayA", "StateHolidayB", "StateHolidayC",
                   "CompetitionOpen", "PromoDecay", "Promo2Decay",
                   "DateTrend", "DateTrendLog",
                   "StateHolidayCLastDecay", "StateHolidayCNextDecay",
                   "Fourier1", "Fourier2", "Fourier3", "Fourier4")

categorical_numeric_features = chain(
    ("Store", "Promo", "Promo2Active", "SchoolHoliday", "DayOfWeek",
     "StateHolidayN", "CompetitionOpen", "StoreTypeN", "AssortmentN",
     "DateTrend", "MDay", "Month", "Year"), decay_features, stairs_features,
    fourier_names
)

In [ ]:
past_date = pd.to_datetime("2000-01-01")
future_date = pd.to_datetime("2099-01-01")

In [ ]:
# Combine data
tftc = pd.concat([train_full, test_csv])
logger.debug("Combined train and test data. Train data shape {0}. Test data shape {1}. Combined data shape {2}".format(
    train_full.shape, test_csv.shape, tftc.shape
))

joined = pd.merge(tftc, store, on='Store', how='inner')
logger.debug("Merged with store data, shape {0}".format(joined.shape))

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
# Add binary feature for each day of week.
# ToDo: add term for Sunday.
for i in range(1, 7):
    joined['DayOfWeek{i}'.format(i=i)] = (joined['DayOfWeek']
                                          .apply(lambda s: 1 if s == i else 0))
    assert check_nulls(joined, 'DayOfWeek{i}'.format(i=i))

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
# Add binary features for StateHoliday categories, and one numerical feature
for i in 'ABC':
    joined['StateHoliday{i}'.format(i=i)] = (joined['StateHoliday']
                                             .apply(lambda s: 1 if s == i.lower() else 0))
    assert check_nulls(joined, 'StateHoliday{i}'.format(i=i))

letter_to_number = {'0': 1, 'a': 2, 'b': 3, 'c': 4}

joined['StateHolidayN'] = joined['StateHoliday'].apply(lambda x: letter_to_number[x])
assert check_nulls(joined, 'StateHolidayN')

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
joined['CompetitionOpen'] = (joined['Date']
                             >= joined['CompetitionOpenDate']).astype(int)
assert check_nulls(joined, 'CompetitionOpen')

In [ ]:
letter_to_number = {'a': 1, 'b': 2, 'c': 3, 'd': 4}

joined['StoreTypeN'] = joined['StoreType'].apply(lambda x: letter_to_number[x])
joined['AssortmentN'] = joined['Assortment'].apply(lambda x: letter_to_number[x])
assert check_nulls(joined, 'StoreTypeN')
assert check_nulls(joined, 'AssortmentN')

In [ ]:
# Represent date offsets
joined['DateTrend'] = ((joined['Date'] - joined['Date'].min())
                       .apply(lambda s: s.days + 1))
assert check_nulls(joined, 'DateTrend')

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
# Binary feature to indicate if Promo2 is active
# ToDo: Check interpretation
month_to_nums = {'Jan,Apr,Jul,Oct': [1, 4, 7, 10],
                 'Feb,May,Aug,Nov': [2, 5, 8, 11],
                 'Mar,Jun,Sept,Dec': [3, 6, 9, 12]}


def is_promo2_active(row):
    if row['Promo2']:
        start_year = row['Promo2SinceYear']
        start_week = row['Promo2SinceWeek']
        interval = row['PromoInterval']
        date = row['Date']
        if ((date.year > start_year)
            or ((date.year == start_year) and (date.week >= start_week))):
            if date.month in month_to_nums[interval]:
                return 1
    return 0


joined['Promo2Active'] = joined[['Promo2', 'Promo2SinceYear',
                                 'Promo2SinceWeek', 'PromoInterval',
                                 'Date']].apply(is_promo2_active, axis=1)
assert check_nulls(joined, 'Promo2Active')

In [ ]:
# Numerical feature for PromoInterval
month_to_nums = {'Jan,Apr,Jul,Oct': 3,
                 'Feb,May,Aug,Nov': 2,
                 'Mar,Jun,Sept,Dec': 4}

joined['PromoIntervalN'] = (joined['PromoInterval']
                            .apply(lambda s: 1 if pd.isnull(s) else month_to_nums[s]))
assert check_nulls(joined, 'PromoIntervalN')

In [ ]:
# Day, month, year
joined['MDay'] = joined['Date'].apply(lambda s: s.day)
joined['Month'] = joined['Date'].apply(lambda s: s.month)
joined['Year'] = joined['Date'].apply(lambda s: s.year)

In [ ]:
# Binary feature for day
for i in range(1, 32):
    joined['MDay{i}'.format(i=i)] = (joined['MDay']
                                     .apply(lambda s: 1 if s == i else 0))

In [ ]:
# Binary feature for month
for i in range(1, 13):
    joined['Month{i}'.format(i=i)] = (joined['Month']
                                      .apply(lambda s: 1 if s == i else 0))

logger.debug("Generated direct features, new shape {0}".format(joined.shape))

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
# Apply transformations grouped by Store

def apply_grouped_by_store(g):
    s = g['Store'].iloc[0]
    logger.debug("Store {s} initial shape {shape}".format(s=s, shape=g.shape))
    g = date_features(g)
    logger.debug("Store {s} after date features shape {shape}".format(s=s, shape=g.shape))
    g = merge_with_fourier_features(g)
    logger.debug("Store {s} final shape {shape}".format(s=s, shape=g.shape))
    return g

In [ ]:
# Merge fourier features
def merge_with_fourier_features(g):
    return pd.merge(g, fourier_features, on='Date')

In [ ]:
def date_features(g):
    g['PromoStarted'] = g['Promo'].diff().apply(lambda s: 1 if s > 0 else 0)
    g['Promo2Started'] = g['Promo2Active'].diff().apply(lambda s: 1 if s > 0 else 0)
    g['Opened'] = g['Open'].diff().apply(lambda s: 1 if s > 0 else 0)
    g['Closed'] = g['Open'].diff().apply(lambda s: 1 if s < 0 else 0)
    g['TomorrowClosed'] = g['Closed'].shift(-1).fillna(0)

    # These are incomplete, NA values filled later.
    g['PromoStartedLastDate'] = g.loc[g['PromoStarted'] == 1, 'Date']
    g['Promo2StartedDate'] = g.loc[g['Promo2Started'] == 1, 'Date']
    g['StateHolidayCLastDate'] = g.loc[g['StateHoliday'] == "c", 'Date']
    g['StateHolidayCNextDate'] = g['StateHolidayCLastDate']
    g['StateHolidayBLastDate'] = g.loc[g['StateHoliday'] == "b", 'Date']
    g['StateHolidayBNextDate'] = g['StateHolidayBLastDate']
    g['StateHolidayANextDate'] = g.loc[g['StateHoliday'] == "a", 'Date']
    g['ClosedLastDate'] = g.loc[g['Closed'] == 1, 'Date']
    g['ClosedNextDate'] = g['ClosedLastDate']
    g['OpenedLastDate'] = g.loc[g['Opened'] == 1, 'Date']
    g['OpenedNextDate'] = g['OpenedLastDate']
    g['LastClosedSundayDate'] = g.loc[(g['Open'] == 0) & (g['DayOfWeek'] == 7), 'Date']

    # Last dates filled with pad
    features = ['PromoStartedLastDate',
                'Promo2StartedDate',
                'StateHolidayCLastDate',
                'StateHolidayBLastDate',
                'ClosedLastDate',
                'OpenedLastDate',
                'LastClosedSundayDate'
                ]
    g[features] = (g[features].fillna(method='pad')
                   .fillna(value=pd.Timestamp('1970-01-01 00:00:00')))
    assert check_nulls(g, features)

    # ToDo: check interpretation
    g['IsClosedForDays'] = (g['Date'] - g['ClosedLastDate']).dt.days

    g['LongOpenLastDate'] = (g.loc[(g['Opened'] == 1)
                                   & (g['IsClosedForDays'] > 5)
                                   & (g['IsClosedForDays'] < 180),
                                   'Date'])

    # Last dates filled with pad
    features = ['LongOpenLastDate',
                ]
    g[features] = (g[features].fillna(method='pad')
                   .fillna(value=pd.Timestamp('1970-01-01 00:00:00')))
    assert check_nulls(g, features)

    #
    g.loc[(g['Open'] == 0) & (g['DayOfWeek'] == 7), 'WasClosedOnSunday'] = 1
    g['WasClosedOnSunday'].fillna(method='pad', limit=6)

    # Next dates filled with backfill
    features = ['StateHolidayCNextDate',
                'OpenedNextDate',
                'ClosedNextDate',
                'StateHolidayBNextDate',
                'StateHolidayANextDate'
                ]
    g[features] = (g[features].fillna(method='backfill')
                   .fillna(value=pd.Timestamp('2020-01-01 00:00:00')))
    assert check_nulls(g, features)

    # ToDo: check interpretation
    g['WillBeClosedForDays'] = (g['OpenedNextDate'] - g['Date']).dt.days

    g['LongClosedNextDate'] = (g.loc[(g['Closed'] == 1)
                                     & (g['WillBeClosedForDays'] > 5)
                                     & (g['WillBeClosedForDays'] < 180),
                                     'Date'])

    # Next dates filled with backfill
    features = ['LongClosedNextDate',
                ]
    g[features] = (g[features].fillna(method='backfill')
                   .fillna(value=pd.Timestamp('2020-01-01 00:00:00')))
    assert check_nulls(g, features)

    return g

In [ ]:
dfj = date_features(joined)

In [ ]:
dfj[['PromoStartedLastDate',
                'Promo2StartedDate',
                'StateHolidayCLastDate',
                'StateHolidayBLastDate',
                'ClosedLastDate',
                'OpenedLastDate',
                'LastClosedSundayDate'
                ]].describe()

In [ ]:
joined = joined.groupby('Store').apply(apply_grouped_by_store)
logger.debug('Expanded with date and fourier features shape {0}'.format(joined.shape))

In [ ]:
joined['Store'].unique()

In [ ]:
pd.isnull(joined[['PromoStartedLastDate',
                'Promo2StartedDate',
                'StateHolidayCLastDate',
                'StateHolidayBLastDate',
                'ClosedLastDate',
                'OpenedLastDate',
                'LastClosedSundayDate'
                ]]).describe()

In [ ]:
__ = pd.isnull(joined).any()
print(__.sum())
__[__]

In [ ]:
def make_decay_features(g, promo_after, promo2_after, holiday_b_before,
                        holiday_c_before, holiday_c_after):
    g['PromoDecay'] = g['Promo'] * np.maximum(
        promo_after - (g['Date'] - g['PromoStartedLastDate']).dt.days
        , 0)
    g['StateHolidayCLastDecay'] = (1 - g['StateHolidayC']) * np.maximum(
        holiday_c_after - (g['Date'] - g['StateHolidayCLastDate']).dt.days
        , 0)
    g['Promo2Decay'] = g['Promo2Active'] * np.maximum(
        promo2_after - (g['Date'] - g['Promo2StartedDate']).dt.days
        , 0)
    g['StateHolidayBNextDecay'] = (1 - g['StateHolidayB']) * np.maximum(
        holiday_b_before - (g['StateHolidayBNextDate'] - g['Date']).dt.days
        , 0)
    g['StateHolidayCNextDecay'] = (1 - g['StateHolidayC']) * np.maximum(
        holiday_c_before - (g['StateHolidayCNextDate'] - g['Date']).dt.days
        , 0)


make_decay_features(joined, promo_after=4, promo2_after=3,
                    holiday_b_before=3, holiday_c_before=15, holiday_c_after=3)
logger.debug("Decay features, new shape {shape}".format(shape=joined.shape))

In [ ]:
def scale_log_features(g, *features):
    for f in features:
        g[f] /= g[f].max()
        g["{0}Log".format(f)] = np.log1p(g[f])
        g["{0}Log".format(f)] /= g["{0}Log".format(f)].max()


scale_log_features(joined, *decay_features, 'DateTrend')
logger.debug("Scale log features, new shape {shape}".format(shape=joined.shape))

In [ ]:
def make_before_stairs(g, *features, days=(2, 3, 4, 5, 7, 14, 28)):
    for f in features:
        before = (g[f] - g['Date']).dt.days
        for d in days:
            g["{0}{1}before".format(f, d)] = before.apply(
                lambda s: 1 if 0 <= s < d else 0)


make_before_stairs(joined, "StateHolidayCNextDate", "StateHolidayBNextDate",
                   "StateHolidayANextDate", "LongClosedNextDate",
                   days=stairs_steps)
logger.debug("Before stairs features, new shape {shape}".format(shape=joined.shape))

In [ ]:
def make_after_stairs(g, *features, days):
    for f in features:
        since = (g['Date'] - g[f]).dt.days
        for d in days:
            g["{0}{1}after".format(f, d)] = since.apply(
                lambda s: 1 if 0 <= s < d else 0)


make_after_stairs(joined, "PromoStartedLastDate", days=(2, 3, 4))
make_after_stairs(joined, "StateHolidayCLastDate", "StateHolidayBLastDate",
                  "Promo2StartedDate", "LongOpenLastDate",
                  days=stairs_steps)
logger.debug("After stairs features, new shape {shape}".format(shape=joined.shape))

logger.debug("Splitting data into train and test, initial shape {shape}".format(shape=joined.shape))
train = joined[(train_range.min() <= joined['Date'])
               & (joined['Date'] <= train_range.max())].drop('Id', axis=1)
logger.debug("Train data shape {shape}".format(shape=train.shape))

test = joined[(test_range.min() <= joined['Date'])
              & (joined['Date'] <= test_range.max())].drop(['Sales', 'Customers'], axis=1)
logger.debug("Test data shape {shape}".format(shape=test.shape))

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
trc = set(train.columns)

In [ ]:
tec = set(test.columns)

In [ ]:
tec.difference(trc)

In [ ]:
example_stores = (
  388, # most typical by svd of Sales time series
  562, # second most typical by svd
  851, # large gap in 2014
  357  # small gap
)

In [ ]:
example_stores = ( 2,4,6,8
)

In [ ]:
small_train = train[train['Store'].apply(lambda s: s in example_stores)]
# small_fold <- make_fold(small_train)

# one_train <- filter(train, Store == 388)
# one_test <- filter(test, Store == 388)

In [ ]:
small_train['Date'].head()

In [ ]:
from collections import namedtuple
dataset = namedtuple('dataset', 'train, test, actual')


def make_fold(train, step=1, predict_interval=6 * 7, step_by=7):
    dates = pd.date_range(train['Date'].min(), train['Date'].max())
    total = dates.shape[0]
    last_train = total - predict_interval - (step - 1) * step_by
    last_train_date = dates[last_train - 1]
    last_predict = last_train + predict_interval
    last_predict_date = dates[last_predict - 1]
    train_set = train[train['Date'] <= last_train_date]
    actual = train[(train['Date'] > last_train_date) & (train['Date'] <=
                                                        last_predict_date)]
    actual['Id'] = np.arange(1, actual.shape[0] + 1)
    test_set = actual.drop('Sales', axis=1)
    return dataset(train=train_set, test=test_set, actual=actual)

In [ ]:
small_fold = make_fold(small_train)

In [ ]:
small_fold[2].shape

In [ ]:
one_train = train[train['Store'] == 388]
one_test = test[test['Store'] == 388]


In [ ]:
A[1]=A[0]+1
A

In [ ]:
A.where((A[0]>=2) & (A[1]<=6))

In [ ]:
A.drop(A.index[(A[0]>=2) & (A[1]<=6)])

In [ ]:
A.columns = ['a','b']

In [ ]:
A.query('a >= 2 & b <= 6 & c <= "2010-04-02"')

In [ ]:
A['c'] = pd.to_datetime('2010-04-01')

In [ ]:
A

In [ ]:
(4,2) > (2,2)

In [ ]:
str(list(test['Store'].unique()))

In [ ]:
train.query('Store in {test_set_stores}'
            .format(test_set_stores=list(test['Store'].unique())
            ))['Store'].unique()

In [ ]:
"a: {0}".format(repr(test['Store'].unique()))

In [ ]:
np.log(A['a'])

In [ ]:
"     aaa    ".strip()

In [ ]:
class Test:
    y = None
    z = None
    @classmethod
    def get_data(cls, x):
        cls.y = x - 1
        cls.z = x + 1
        
    @classmethod
    def show(cls):
        print("y: {y}, z: {z}".format(y=cls.y, z=cls.z))

In [ ]:
Test.get_data(8)

In [ ]:
Test.show()

In [ ]:
A

In [ ]:
@allow_modifications(False)
def expandA(A):
    #     A = A.query('a > 2')
    A['d']=0

In [ ]:
expandA(A)

In [ ]:
del A['d']

In [ ]:
A

In [ ]:
def allow_modifications(allow):
    def decorate(func):
        func_name = func.__name__

        def wrapper(_df, *args, **kwargs):
            _df1 = _df.copy()
            return_val = func(_df, *args, **kwargs)
            try:
                assert (allow and not _df1.equals(_df)) or (not allow and _df1.equals(_df))
            except AssertionError:
                if allow:
                    logger.warning('!{f} does not modify dataframe!'.format(f=func_name))
                else:
                    logger.warning('!{f} is modifying dataframe!'.format(f=func_name))
            else:
                if not allow:
                    logger.debug('{f} does not modify dataframe'.format(f=func_name))
                else:
                    logger.debug('{f} is modifying dataframe'.format(f=func_name))
            return return_val
        return wrapper

    return decorate